In [2]:
import sagemaker
import boto3

sess = sagemaker.Session()

bucket = 'kaggle-writing-student'
key_dataset = 'dataset'
output_processed_data = 'processed_dataset'

raw_input_data_s3_uri = 's3://{}/{}/'.format(bucket, key_dataset)
output_data_s3_uri = 's3://{}/{}/'.format(bucket, output_processed_data)
                     
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name="sagemaker", region_name=region)
s3 = boto3.Session().client(service_name="s3", region_name=region)

In [3]:
raw_input_data_s3_uri, output_data_s3_uri

('s3://kaggle-writing-student/dataset/',
 's3://kaggle-writing-student/processed_dataset/')

In [4]:
train_split_percentage = 0.8
validation_split_percentage = 0.1
test_split_percentage = 0.1

In [5]:
# Accepted instances are currently (according to error message):
# [ml.r5.12xlarge, ml.m5.4xlarge, ml.p2.xlarge, ml.m4.16xlarge, ml.r5.24xlarge, ml.t3.xlarge,
#  ml.r5.16xlarge, ml.m5.large, ml.p3.16xlarge, ml.p2.16xlarge, ml.c4.2xlarge, ml.c5.2xlarge, 
#  ml.c4.4xlarge, ml.c5.4xlarge, ml.g4dn.xlarge, ml.g4dn.12xlarge, ml.g4dn.2xlarge, ml.c4.8xlarge, 
#  ml.g4dn.4xlarge, ml.c5.9xlarge, ml.g4dn.16xlarge, ml.c5.xlarge, ml.c4.xlarge, ml.g4dn.8xlarge, 
#  ml.t3.2xlarge, ml.t3.medium, ml.c5.18xlarge, ml.r5.2xlarge, ml.p3.2xlarge, ml.m5.xlarge, ml.m4.10xlarge, 
# ml.r5.4xlarge, ml.m5.12xlarge, ml.m4.xlarge, ml.t3.large, ml.m5.24xlarge, ml.m4.2xlarge, ml.m5.2xlarge, 
# ml.p2.8xlarge, ml.r5.8xlarge, ml.r5.xlarge, ml.r5.large, ml.p3.8xlarge, ml.m4.4xlarge]

In [6]:
processing_instance_type = "ml.m5.large"
processing_instance_count = 1

In [ ]:
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput

processor = SKLearnProcessor(
    framework_version="0.23-1",
    role=role,
    instance_type=processing_instance_type,
    instance_count=processing_instance_count,
    env={"AWS_DEFAULT_REGION": region},
    max_runtime_in_seconds=1800,
)

In [ ]:
processor.run(code='processing_script.py',
              inputs=[ProcessingInput(
                  source=raw_input_data_s3_uri,
                  destination='/opt/ml/processing/input_data/')],
              outputs=[ProcessingOutput(
                  output_name='processed_data',
                  source='/opt/ml/processing/processed_data/',
                  destination=output_data_s3_uri,
                  s3_upload_mode='EndOfJob',
                  )],
              arguments=[
                  "train-split-percentage",
                  str(train_split_percentage),
                  "validation-split-percentage",
                  str(validation_split_percentage),
                  "test-split-percentage",
                  str(test_split_percentage),
              ],
              logs=True,
              wait=False,
              )

In [9]:
scikit_processing_job_name = processor.jobs[-1].describe()["ProcessingJobName"]
print(scikit_processing_job_name)

sagemaker-scikit-learn-2022-02-23-10-49-32-546


In [10]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/sagemaker/home?region={}#/processing-jobs/{}">Processing Job</a></b>'.format(
            region, scikit_processing_job_name
        )
    )
)

In [11]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://console.aws.amazon.com/cloudwatch/home?region={}#logStream:group=/aws/sagemaker/ProcessingJobs;prefix={};streamFilter=typeLogStreamPrefix">CloudWatch Logs</a> After About 5 Minutes</b>'.format(
            region, scikit_processing_job_name
        )
    )
)

In [12]:
from IPython.core.display import display, HTML

display(
    HTML(
        '<b>Review <a target="blank" href="https://s3.console.aws.amazon.com/s3/buckets/{}/{}/?region={}&tab=overview">S3 Output Data</a> After The Processing Job Has Completed</b>'.format(
            bucket, output_processed_data, region
        )
    )
)

In [ ]:
running_processor = sagemaker.processing.ProcessingJob.from_processing_name(
    processing_job_name=scikit_processing_job_name, sagemaker_session=sess
)

processing_job_description = running_processor.describe()

print(processing_job_description)

In [14]:
running_processor.wait(logs=False)

...............................................................................!

In [15]:
student_writing_output_data_s3_uri = output_data_s3_uri

In [16]:
%store student_writing_output_data_s3_uri

Stored 'student_writing_output_data_s3_uri' (str)


# TODO: prepare the script to include the Feature Store part

(Currently preparing the pipeline)